### 1. Import dependencies and set directory

In [1]:
# Import dependencies
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import rpy2.rinterface
import rpy2_arrow.pyarrow_rarrow as pyra
import pyarrow.feather as feather

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\Sang\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
%pwd

'C:\\Users\\Sang\\aus_rain_cloud\\notebooks'

In [4]:
%cd /Users/Sang/

C:\Users\Sang


### 2. Downloading the data

In [5]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [6]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]             

In [7]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

Wall time: 1min 6s


In [8]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

Wall time: 23.4 s


### 3. Combining data CSVs

In [9]:
%%time
%%memit

files = glob.glob('figshareairline/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0,)
                .assign(model = re.findall(r'(?<=\\)(.*)(?=_)', file)[0].split("_")[0]) for file in files)
              )

df.to_csv("figshareairline/combined_data.csv")

peak memory: 7000.18 MiB, increment: 6724.74 MiB
Wall time: 8min 15s


### 4. EDA in R

In [10]:
%%R
# call library
library("arrow")
library("tidyverse")
library("dplyr")

R[write to console]: -- Attaching packages --------------------------------------- tidyverse 1.3.0 --

R[write to console]: <U+221A> ggplot2 3.3.3     <U+221A> purrr   0.3.4
<U+221A> tibble  3.0.5     <U+221A> dplyr   1.0.3
<U+221A> tidyr   1.1.2     <U+221A> stringr 1.4.0
<U+221A> readr   1.4.0     <U+221A> forcats 0.5.1

R[write to console]: -- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



### 4.1. Create feather format

In [11]:
%%time
%%memit
dataset = ds.dataset("figshareairline/combined_data.csv", format="csv")
table = dataset.to_table()

peak memory: 7655.61 MiB, increment: 3872.68 MiB
Wall time: 33.9 s


In [12]:
%%time
%%memit
# create feather file
feather.write_feather(table, 'figshareairline/rain.feather')

peak memory: 7654.92 MiB, increment: 0.00 MiB
Wall time: 4.77 s


In [13]:
%%sh
# check the data size
du -sh figshareairline/combined_data.csv
du -sh figshareairline/rain.feather

5.7G	figshareairline/combined_data.csv
1.1G	figshareairline/rain.feather


### 4.2. Perform EAD with various file format

In [14]:
%%time
%%memit
rain_df = pd.read_csv("figshareairline/combined_data.csv")

peak memory: 9818.12 MiB, increment: 2166.01 MiB
Wall time: 1min 23s


In [15]:
%%time
%%memit
# loading the arrow dataframe
rain_table = pyra.converter.py2rpy(table)

5756
rarrow.ChunkedArray: 0.030597686767578125
5756
rarrow.ChunkedArray: 0.055944204330444336
5756
rarrow.ChunkedArray: 0.023988008499145508
5756
rarrow.ChunkedArray: 0.02599787712097168
5756
rarrow.ChunkedArray: 0.04196524620056152
5756
rarrow.ChunkedArray: 0.04106497764587402
5756
rarrow.ChunkedArray: 0.02396988868713379
peak memory: 5449.47 MiB, increment: 332.48 MiB
Wall time: 28 s


In [16]:
%%time
%%R -i rain_table
start_time <- Sys.time()
result_arrow <- rain_table %>% collect() %>% count(model)
print(class(rain_table %>% collect()))
end_time <- Sys.time()
print(result_arrow)
print(end_time - start_time)

[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 28 x 2
   model                  n
 * <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 18 more rows
Time difference of 26.77976 secs
Wall time: 27 s


In [17]:
# I thinks there was no huge difference between arrow and feather but feather takes longer time than arrow on my side. 
# %%time
# %%R
# rain_table_feather <- arrow::read_feather("figshareairline/rain.feather")

In [18]:
# %%time
# %%R -i rain_table_feather
# start_time <- Sys.time()
# result_feather <- rain_table_feather %>% count(model)
# end_time <- Sys.time()
# print(result_feather)
# print(end_time - start_time)

**why you chose this approach over others?**
>- Faster than other format in terms of reading and writing.
>- `Pandas exchange` did not work due to memory issue
>- `feather` did took so much time and it did not work. The size of feather file is less than that of csv file but there was memory issue in local computer.
>- `Parquet` has slower speed than `feather` in terms of reading and writing.  